##Mounting drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/Colab Notebooks/Data_mining/Lab1/Notebooks
!ls

/content/drive/My Drive/Colab Notebooks/Data_mining/Lab1/Notebooks
apriori.ipynb  fp_Growth.ipynb


##Importing Libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import math
import csv
import time
import re
import tracemalloc

# Node class

In [4]:
class Node:
  def __init__(self, parent= None, val = None):
    self.parent = parent
    self.val = val
    self.children = {}
    self.supCnt = 0
    self.next = None
    if parent == None:
      self.level = 0
    else:
      self.level = parent.level+1
      parent.insertChild(self)

  def insertChild(self, node):
    self.children[node.val] = node

  def deleteChild(self, node):
    del self.children[node.val]
  

# FP Growth implementation

In [5]:
def sortTransactions(transactions):
  for i in range(len(transactions)):
    transactions[i] = np.unique(transactions[i])
  return transactions

In [6]:
def constructFpTree(transactions, transactionFrequency, minSup):
  headerTable = {}
  freqPerItem = {}
  
  root = Node()

  for i in range(len(transactions)):
    transaction = transactions[i]
    for item in transaction:
      if item in freqPerItem:
        freqPerItem[item]+=transactionFrequency[i]
      else:        
        freqPerItem[item]=transactionFrequency[i]
  
  itemKeys = list(freqPerItem)
  for i in range(len(itemKeys)):
    if freqPerItem[itemKeys[i]]<minSup:
      del freqPerItem[itemKeys[i]]

  for item in freqPerItem:
    headerTable[item] = {
                'frequency': freqPerItem[item],
                'next': None
              }
  

  if not headerTable:
    return None, None
  
  for ind in range(len(transactions)):
    transaction = transactions[ind]
    filteredTransaction = [item for item in transaction if item in headerTable]
    filteredTransaction = sorted(filteredTransaction, key= lambda i : headerTable[i]['frequency'], reverse=True)
    
    curNode = root
    for item in filteredTransaction:
      curNode = insertItemToTree(item, curNode, headerTable, transactionFrequency[ind])
      

  return root, headerTable


In [7]:

def insertItemToTree(item, node,  headerTable, transactionFrequency):
  if item in node.children:
    node.children[item].supCnt += transactionFrequency
  else:
    newNode = Node(node, item)
    newNode.supCnt += transactionFrequency
    updateHeaderTable(item, newNode, headerTable)

  return node.children[item]

def updateHeaderTable(item, node, headerTable):
  if headerTable[item]['next']== None:
    headerTable[item]['next'] = node
  else:
    newNode = headerTable[item]['next']
    while newNode.next != None:
        newNode = newNode.next
    newNode.next = node

In [8]:
def frequentItemsetMining(headerTable, minSup, bucket):
  global frequentItemsets

  totalFrequentItemsets = 0
  
  headerItems = list(headerTable)
  headerItems.sort(key=lambda item: headerTable[item]['frequency'])

  for item in headerItems:  
    newfrequentItemset = bucket.copy()
    newfrequentItemset = np.append(newfrequentItemset, item)
    totalFrequentItemsets += 1
    frequentItemsets.append(newfrequentItemset)
    conditionalDatabase, frequency = generateConditionalDatabase(item, headerTable) 
    conditionalFpTree, conditionalHeaderTable = constructFpTree(conditionalDatabase, frequency, minSup) 
    if conditionalHeaderTable != None:
      totalFrequentItemsets += frequentItemsetMining(conditionalHeaderTable, minSup, newfrequentItemset)
  return totalFrequentItemsets


In [9]:
def generateConditionalDatabase(item, headerTable):
  node = headerTable[item]['next'] 
  ansestorPaths = []
  frequency = np.array([], dtype= int)
  while node != None:
    ansestorPath = []
    getAnsestorPath(node, ansestorPath)  
    ansestorPath = np.array(ansestorPath)
    if len(ansestorPath) > 1:
      ansestorPaths.append(ansestorPath[1:])
      frequency= np.append(frequency, node.supCnt)

    node = node.next  
  return ansestorPaths, frequency

def getAnsestorPath(node, ansestorPath):
  if node.parent != None:
    ansestorPath.append( node.val)
    getAnsestorPath(node.parent, ansestorPath)

In [10]:
frequentItemsets = []
def runFpGrowth(transactions, minSup):
  global frequentItemsets
  frequentItemsets = []
  transactionFrequency = np.ones((len(transactions),))
  root, headerTable = constructFpTree(transactions, transactionFrequency, minSup)
  totalFrequentItemsets = frequentItemsetMining(headerTable, minSup, np.array([]))
  return totalFrequentItemsets



# Dataset Load

In [11]:
def loadDataset(datasetName, isNumeric = False):
  filepath = '/content/drive/My Drive/Colab Notebooks/Data_mining/Lab1/Datasets/'+datasetName
  # filepath = '/content/drive/MyDrive/Colab Notebooks/Data Mining Assignments/{}'.format(datasetName)
  transactions = []
  # allItems = np.array([])
  totalTransactions = 0
  with open(filepath) as f:
    for i, line in enumerate(f):
      totalTransactions+=1
      transaction = np.unique(line.strip().split(' ')) 
      if isNumeric:
        # allItems = np.append(allItems, np.array(transaction, dtype = int ))
        transactions.append(np.array(transaction, dtype = int))
      else:
        # allItems = np.append(allItems, transaction)
        transactions.append(transaction)
      
  
  # print(transactions)
  # print(allItems)

  return transactions, totalTransactions

# transactions, allItems = loadDataset('sampleDataset_1.csv', False)
# print(transactions)
# print(allItems)

In [12]:
transactions, totalTransactions  = loadDataset('sampleDataset_1.csv', False)
# transactions, totalTransactions = loadDataset('sampleDataset-1.csv', False)
print(transactions)
totalFrequentItem = runFpGrowth(transactions, 3)
print('Total frequent itemsets genereted = ',totalFrequentItem)


transactions, totalTransactions = loadDataset('sampleDataset_2.csv', False)
# transactions, totalTransactions = loadDataset('sampleDataset-2.csv', False)
print(transactions)
totalFrequentItem = runFpGrowth(transactions, 2)
print('Total frequent itemsets genereted = ',totalFrequentItem)

[array(['a', 'c', 'd', 'f', 'g', 'i', 'm', 'p'], dtype='<U1'), array(['a', 'b', 'c', 'f', 'l', 'm', 'o'], dtype='<U1'), array(['b', 'f', 'h', 'j', 'o'], dtype='<U1'), array(['b', 'c', 'k', 'p', 's'], dtype='<U1'), array(['a', 'c', 'e', 'f', 'l', 'm', 'n', 'p'], dtype='<U1')]
Total frequent itemsets genereted =  18
[array(['I1', 'I2', 'I5'], dtype='<U2'), array(['I2', 'I4'], dtype='<U2'), array(['I2', 'I3'], dtype='<U2'), array(['I1', 'I2', 'I4'], dtype='<U2'), array(['I1', 'I3'], dtype='<U2'), array(['I2', 'I3'], dtype='<U2'), array(['I1', 'I3'], dtype='<U2'), array(['I1', 'I2', 'I3', 'I5'], dtype='<U2'), array(['I1', 'I2', 'I3'], dtype='<U2')]
Total frequent itemsets genereted =  13


# Datasets

In [19]:
def printAndSave(datasetName,timeCllection, memoryCllection, minSupCnts):
  timeStr = ''
  memoryStr=''
  for i in range(len(timeCllection)):
    if i==0:
      timeStr+='FP-Growth,'+str(timeCllection[i])
    elif i == len(timeCllection)-1:
      timeStr+=','+str(timeCllection[i])+'\n'
    else:
      timeStr+=','+str(timeCllection[i])

  for i in range(len(memoryCllection)):
    if i==0:
      memoryStr+='FP-Growth,'+str(memoryCllection[i]/ 10**6)
    elif i == len(memoryCllection)-1:
      memoryStr+=','+str(memoryCllection[i]/ 10**6)+'\n'
    else:
      memoryStr+=','+str(memoryCllection[i]/ 10**6)

  print('timeString: ',timeStr)
  print('memoryString: ',memoryStr)
  filepath = '/content/drive/My Drive/Colab Notebooks/Data_mining/Lab1/Output/timeCollection_'+datasetName+'.csv'
  with open(filepath,'a') as f:
    f.write(timeStr)
  filepath = '/content/drive/My Drive/Colab Notebooks/Data_mining/Lab1/Output/memoryCollection_'+datasetName+'.csv'
  with open(filepath,'a') as f:
    f.write(memoryStr)

## Chess

In [14]:
timeCllection1= []
memoryCllection1= []
transactions, totalTransactions = loadDataset('chess.dat', True)
minRelativeSupCnts = np.linspace(0.95, 0.8, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:  

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runFpGrowth(transactions, minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection1.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection1.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[3037. 2917. 2797. 2677. 2557.]
Total frequent itemsets genereted =  77
0.2860240936279297
Current memory usage is 0.107335MB; Peak was 0.134641MB
Total frequent itemsets genereted =  389
0.3362853527069092
Current memory usage is 0.287036MB; Peak was 0.314901MB
Total frequent itemsets genereted =  1386
0.49938154220581055
Current memory usage is 0.78591MB; Peak was 0.812949MB
Total frequent itemsets genereted =  3674
0.6554417610168457
Current memory usage is 1.210221MB; Peak was 1.237293MB
Total frequent itemsets genereted =  8227
1.0625207424163818
Current memory usage is 2.271164MB; Peak was 2.387974MB


In [20]:
# printAndSave('chess',timeCllection1, memoryCllection1, np.linspace(0.95, 0.75, 5))

timeString:  FP-Growth,0.2860240936279297,0.3362853527069092,0.49938154220581055,0.6554417610168457,1.0625207424163818

memoryString:  FP-Growth,0.134641,0.314901,0.812949,1.237293,2.387974



## Mushroom

In [15]:
timeCllection2= []
memoryCllection2= []
transactions, totalTransactions = loadDataset('mushroom.dat', True)
minRelativeSupCnts = np.linspace(0.60, 0.40, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:  

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runFpGrowth(transactions, minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection2.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection2.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[4875. 4469. 4062. 3656. 3250.]
Total frequent itemsets genereted =  51
0.5012552738189697
Current memory usage is 0.067751MB; Peak was 0.133209MB
Total frequent itemsets genereted =  99
0.5017781257629395
Current memory usage is 0.105743MB; Peak was 0.171753MB
Total frequent itemsets genereted =  153
0.5468237400054932
Current memory usage is 0.1446MB; Peak was 0.210673MB
Total frequent itemsets genereted =  329
0.5926194190979004
Current memory usage is 0.345836MB; Peak was 0.412197MB
Total frequent itemsets genereted =  565
0.6407942771911621
Current memory usage is 0.506364MB; Peak was 0.572749MB


In [21]:
# printAndSave('mushroom',timeCllection2, memoryCllection2,  np.linspace(0.60, 0.40, 5))

timeString:  FP-Growth,0.5012552738189697,0.5017781257629395,0.5468237400054932,0.5926194190979004,0.6407942771911621

memoryString:  FP-Growth,0.133209,0.171753,0.210673,0.412197,0.572749



## Retail

In [16]:
timeCllection3= []
memoryCllection3= []
transactions, totalTransactions = loadDataset('retail.dat', True)
minRelativeSupCnts = np.linspace(0.005, 0.003, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:  

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runFpGrowth(transactions, minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection3.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection3.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[441. 397. 353. 309. 265.]
Total frequent itemsets genereted =  580
14.195393323898315
Current memory usage is 53.892644MB; Peak was 55.60921MB
Total frequent itemsets genereted =  703
16.86274480819702
Current memory usage is 64.355395MB; Peak was 66.03717MB
Total frequent itemsets genereted =  831
19.76856279373169
Current memory usage is 71.683771MB; Peak was 73.393074MB
Total frequent itemsets genereted =  1075
21.18426752090454
Current memory usage is 84.310356MB; Peak was 85.878354MB
Total frequent itemsets genereted =  1393
24.57249689102173
Current memory usage is 100.312508MB; Peak was 102.014482MB


In [25]:
# printAndSave('retail',timeCllection3, memoryCllection3,  np.linspace(0.005, 0.003, 5))

timeString:  FP-Growth,14.195393323898315,16.86274480819702,19.76856279373169,21.18426752090454,24.57249689102173

memoryString:  FP-Growth,55.60921,66.03717,73.393074,85.878354,102.014482



## T10I4D100K

In [17]:
timeCllection4= []
memoryCllection4= []
transactions, totalTransactions = loadDataset('T10I4D100K.dat', True)
minRelativeSupCnts = np.linspace(0.04, 0.02, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:  

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runFpGrowth(transactions, minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection4.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection4.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[4000. 3501. 3000. 2500. 2000.]
Total frequent itemsets genereted =  26
3.6304988861083984
Current memory usage is 3.179615MB; Peak was 4.292961MB
Total frequent itemsets genereted =  40
5.18127179145813
Current memory usage is 10.167717MB; Peak was 11.596265MB
Total frequent itemsets genereted =  60
10.70578384399414
Current memory usage is 25.05575MB; Peak was 26.696721MB
Total frequent itemsets genereted =  107
40.50693416595459
Current memory usage is 70.369895MB; Peak was 72.227305MB
Total frequent itemsets genereted =  155
75.08197045326233
Current memory usage is 115.120383MB; Peak was 116.995889MB


In [23]:
# printAndSave('T10I4D100K',timeCllection4, memoryCllection4, np.linspace(0.04, 0.02, 5))

timeString:  FP-Growth,3.6304988861083984,5.18127179145813,10.70578384399414,40.50693416595459,75.08197045326233

memoryString:  FP-Growth,4.292961,11.596265,26.696721,72.227305,116.995889



## kosarak

In [18]:
timeCllection5= []
memoryCllection5= []
transactions, totalTransactions = loadDataset('kosarak.dat', True)
minRelativeSupCnts = np.linspace(0.15, 0.05, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:  

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runFpGrowth(transactions, minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection5.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection5.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[148501. 123751.  99001.  74251.  49501.]
Total frequent itemsets genereted =  7
26.38734769821167
Current memory usage is 0.018829MB; Peak was 11.586717MB
Total frequent itemsets genereted =  9
26.446165084838867
Current memory usage is 0.01835MB; Peak was 11.584653MB
Total frequent itemsets genereted =  9
26.365927934646606
Current memory usage is 0.017206MB; Peak was 11.584677MB
Total frequent itemsets genereted =  16
26.924052238464355
Current memory usage is 0.065263MB; Peak was 11.584653MB
Total frequent itemsets genereted =  33
27.37964415550232
Current memory usage is 0.445971MB; Peak was 11.584269MB


In [24]:
# printAndSave('kosarak',timeCllection5, memoryCllection5, np.linspace(0.15, 0.05, 5))

timeString:  FP-Growth,26.38734769821167,26.446165084838867,26.365927934646606,26.924052238464355,27.37964415550232

memoryString:  FP-Growth,11.586717,11.584653,11.584677,11.584653,11.584269

